In [1]:
import networkx as nx
from scipy.optimize import curve_fit
import scipy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import osmnx as ox
from sklearn.cluster import KMeans
from sklearn import metrics
from itertools import cycle
import matplotlib.cm as cm
from shapely import geometry
from scipy.spatial import ConvexHull
from sklearn.metrics import mean_squared_error
import geopandas as gpd
from shapely.geometry import Polygon
from shapely import geometry, ops
from scipy.spatial import Voronoi, voronoi_plot_2d

In [2]:
def get_Graph(city):
    nodes, edges = pd.read_csv(f"{city}_nodes.csv", index_col =[0]), pd.read_csv(f"./{city}_edges.csv", index_col = [0,1,2])
    edges["highway"] = edges.highway.map(lambda x: x[0] if isinstance(x, list) else x)
    others = ["crossing", "living_street", "unclassified", "disused", "busway", "escape", "road"]
    edges["highway"] = edges.highway.map(lambda x: "other" if x  in others else x)
    edges["Cost of returnal"] = edges["Cost of returnal"].fillna(0)
    s = gpd.GeoSeries.from_wkt(nodes.geometry)
    nodes = gpd.GeoDataFrame(data = nodes, geometry = s)
    nodes = nodes.set_crs('epsg:4326', allow_override=True)
    s = gpd.GeoSeries.from_wkt(edges.geometry)
    edges = gpd.GeoDataFrame(data = edges, geometry = s)
    edges = edges.set_crs('epsg:4326', allow_override=True)

    G = ox.graph_from_gdfs(nodes, edges)
    H = nx.MultiDiGraph()
    H.add_nodes_from(sorted(G.nodes(data=True)))
    H.add_edges_from(G.edges(data=True))
    H.graph["crs"] = G.graph["crs"]
    return H, nodes, edges

In [3]:
city_list_full = ["São Paulo", # 0 
                  "Rio de Janeiro", # 1
                   "Buenos Aires", # 2
                   "Atlanta", # 3
                   "London", # 4
                   "Barcelona", # 5
                   "Madrid", # 6
                   "Paris", # 7
                   "Berlin", # 8
                   "Amsterdam",# 9
                   "Beijing",# 10
                   ]

In [13]:
for city in city_list_full:
    G, nodes, edges = get_Graph(city)
    series = {"COR": edges["Cost of returnal"],
            "lat": edges.representative_point().x,
            "lon": edges.representative_point().y}
    df = pd.DataFrame(series).reset_index()
    # print(sum(df["COR"] == 0)/len(df))
    df = df.drop(columns = ["u", "v", "key"])
    df.replace(0, 100, inplace=True)
    df.to_csv(f"centroids/{city}_CORpoints.csv", index_label = "edge")